In [ ]:
import os
from datetime import datetime

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import xgboost as xgb
from catboost import CatBoostClassifier
from matplotlib.lines import Line2D
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay, accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, LearningCurveDisplay
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import warnings

warnings.filterwarnings('ignore')

#### Wczytanie przygotowanych danych

In [ ]:
train = pd.read_csv(os.path.join('prepared', 'train.csv'))
test = pd.read_csv(os.path.join('prepared', 'test.csv'))

salary_category_encoder = LabelEncoder().fit(train['salary_category'])

test_obs = test['obs']
test.drop(columns=['obs'], inplace=True)

my_categories = ['job_title', 'feature_1', 'job_state', 'job_region']
train[my_categories] = train[my_categories].astype('category')
test[my_categories] = test[my_categories].astype('category')

In [ ]:
train.info()

In [ ]:
test.info()

### <center>Proces nauki, walidacji, testowania</center>

In [ ]:
X = train.drop(columns=['salary_category']).reset_index(drop=True)
y = salary_category_encoder.transform(train['salary_category'])

class_labels = np.unique(y)

test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

#### LightGBM

In [ ]:
def custom_accuracy(preds, eval_data):
    y_true = eval_data.get_label()
    y_pred = preds.argmax(axis=1)
    return 'accuracy', accuracy_score(y_true, y_pred), True

def objective_lgb(trial):
    dataset = lgb.Dataset(X, y, params={
        'max_bin': trial.suggest_int('max_bin', 32, 256),
        'feature_pre_filter': False
    })

    params = {
        'objective': 'multiclass',
        'num_class': len(np.unique(y)),
        'boosting': 'gbdt',
        'class_weight': 'balanced',
        'n_jobs': -1,
        'metric': 'None',
        'random_state': 81,
        'verbose': -1,
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'num_leaves': trial.suggest_int('num_leaves', 8, 64),
        'max_depth': trial.suggest_int('max_depth', 8, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 50),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
    }

    cv_results = lgb.cv(
        params=params,
        train_set=dataset,
        folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        feval=custom_accuracy,
        metrics='None',
        return_cvbooster=False,
    )
    mean_accuracy = np.mean(cv_results['valid accuracy-mean'])
    return mean_accuracy

study_lgb = optuna.create_study(direction='maximize', study_name='EngineerSalaryLightGBM', sampler=optuna.samplers.TPESampler())
study_lgb.optimize(objective_lgb, n_trials=100)

In [ ]:
best_params = study_lgb.best_params

best_params

In [ ]:
standard_params = {
    'objective': 'multiclass',
    'class_weight': 'balanced',
    'num_class': len(np.unique(y)),
    'boosting': 'gbdt',
    'n_jobs': -1,
    'metric': 'None',
    'random_state': 81,
    'verbose': -1,
}

best_params = {**standard_params, **best_params}

lightgbm = lgb.LGBMClassifier(**best_params).fit(X_train, y_train)

#### Wizualizacja z optuną

In [ ]:
optuna.visualization.plot_optimization_history(study_lgb)

In [ ]:
optuna.visualization.plot_slice(study_lgb)

In [ ]:
optuna.visualization.plot_param_importances(study_lgb)

In [ ]:
compare_skf = pd.DataFrame(data={'lgb': [], 'xgb': [], 'cat': []})

#### Jak LightGBM wypada w poszczególnych foldach

In [ ]:
# lightgbm = lgb.LGBMClassifier(**{
#     "boosting_type": 'gbdt',
#     "class_weight": None,
#     "colsample_bytree": 0.9333584928887282,
#     "importance_type": 'split',
#     "learning_rate": 0.23332896051177257,
#     "max_depth": 12,
#     "min_child_samples": 20,
#     "min_child_weight": 0.001,
#     "min_split_gain": 0.0,
#     "n_estimators": 142,
#     "n_jobs": -1,
#     "num_leaves": 34,
#     "objective": 'multiclass',
#     "random_state": 81,
#     "reg_alpha": 0.0,
#     "reg_lambda": 1,
#     "subsample": 0.9720591168744056,
#     "subsample_for_bin": 200000,
#     "subsample_freq": 0,
#     "num_class": 3,
#     "boosting": 'gbdt',
#     "metric": None,
#     "verbose": -1,
#     "min_data_in_leaf": 6,
# })

In [ ]:
folds = []
stratifiedkfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i, (train_idx, val_idx) in enumerate(stratifiedkfold.split(X, y)):
    X_train_fold, X_val_fold = X.loc[train_idx, :], X.loc[val_idx, :]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    lightgbm.fit(X_train_fold, y_train_fold)
    folds.append(accuracy_score(y_val_fold, lightgbm.predict(X_val_fold)) * 100)

compare_skf['lgb'] = folds

In [ ]:
lightgbm.fit(X_train, y_train)
lightgbm_pred = lightgbm.predict(X_test)
print(classification_report(y_test, lightgbm_pred, target_names=salary_category_encoder.inverse_transform(np.unique(y_train))))

#### Istotność cech dla LightGBM

In [ ]:
lightgbm_importance = pd.DataFrame({
    'feature': lightgbm.feature_name_,
    'importance': lightgbm.feature_importances_
}).sort_values(by='importance', ascending=False).head(10)

plt.figure(figsize=(12, 8))
sns.set_style('whitegrid')

ax = sns.barplot(
    data=lightgbm_importance,
    x='importance',
    y='feature',
    hue='feature',
    legend=False,
    palette='viridis'
)

for i in ax.containers:
    ax.bar_label(i, fmt='%g', label_type='edge', fontsize=10, padding=3)

plt.title('Top 10 najważniejszych cech dla LightGBM', fontsize=16)
plt.xlabel('Wartość istotności', fontsize=12)
plt.ylabel('Cecha', fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

#### Drzewo decyzyjne w LightGBM

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
lgb.plot_tree(
    lightgbm,
    ax=ax,
    show_info='data_percentage',
    precision=2,
    orientation='vertical',
    dpi=500
)

#### XGBoost

In [ ]:
def define_xgb(trial):
    params = {
        'tree_method': 'hist',
        'enable_categorical': True,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'max_leaves': trial.suggest_int('max_leaves', 8, 32),
        'gamma': trial.suggest_float('gamma', 0.5, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1e-1, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 1, log=True),
        'max_bin': trial.suggest_int('max_bin', 32, 256),
        'n_jobs': -1,
        'objective': 'multiclass',
        'num_class': len(np.unique(y)),
        'random_state': 42
    }
    return xgb.XGBClassifier(**params)

def objective_xgb(trial):
    model = define_xgb(trial)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=skf, n_jobs=-1, scoring='accuracy')
    return scores.mean()

study_xgb = optuna.create_study(direction='maximize', study_name='EngineerSalaryXGBoost', sampler=optuna.samplers.TPESampler())
study_xgb.optimize(objective_xgb, n_trials=100)

xgBoost = define_xgb(study_xgb.best_trial).fit(X_train, y_train)

In [ ]:
study_xgb.best_params

#### Wizualizacji procesu optymalizacji hiperparametrów

In [ ]:
optuna.visualization.plot_optimization_history(study_xgb)

In [ ]:
optuna.visualization.plot_slice(study_xgb)

In [ ]:
optuna.visualization.plot_param_importances(study_xgb)

#### Jak wypada XGBoost w poszczególnych foldach

In [ ]:
# xgBoost = xgb.XGBClassifier(**{
#     "objective": 'multi:softprob',
#     "use_label_encoder": None,
#     "base_score": None,
#     "booster": None,
#     "callbacks": None,
#     "colsample_bylevel": None,
#     "colsample_bynode": None,
#     "colsample_bytree": None,
#     "early_stopping_rounds": None,
#     "enable_categorical": True,
#     "eval_metric": None,
#     "feature_types": None,
#     "gamma": 0.6551776163530296,
#     "gpu_id": None,
#     "grow_policy": None,
#     "importance_type": None,
#     "interaction_constraints": None,
#     "learning_rate": 0.07315523763489679,
#     "max_bin": 219,
#     "max_cat_threshold": None,
#     "max_cat_to_onehot": None,
#     "max_delta_step": None,
#     "max_depth": 10,
#     "max_leaves": 23,
#     "min_child_weight": None,
#     "monotone_constraints": None,
#     "n_estimators": 118,
#     "n_jobs": -1,
#     "num_parallel_tree": None,
#     "predictor": None,
#     "random_state": 42,
#     "reg_alpha": 0.013705730913738874,
#     "reg_lambda": 0.012219537217204616,
#     "sampling_method": None,
#     "scale_pos_weight": None,
#     "subsample": 0.9988236950762315,
#     "tree_method": 'hist',
#     "validate_parameters": None,
#     "verbosity": None,
#     "num_class": 3,
# })

In [ ]:
folds = []
stratifiedkfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i, (train_idx, val_idx) in enumerate(stratifiedkfold.split(X, y)):
    X_train_fold, X_val_fold = X.loc[train_idx, :], X.loc[val_idx, :]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    xgBoost.fit(X_train_fold, y_train_fold)
    folds.append(accuracy_score(y_val_fold, xgBoost.predict(X_val_fold)) * 100)

compare_skf['xgb'] = folds

In [ ]:
xgBoost.fit(X_train, y_train)
xgBoost_pred = xgBoost.predict(X_test)

print(classification_report(y_test, xgBoost_pred, target_names=salary_category_encoder.inverse_transform(np.unique(y_train))))

#### Istotność cech dla XGBoost

In [ ]:
importance_dict = xgBoost.get_booster().get_score(importance_type='weight')

xgBoost_importance = pd.DataFrame({
    'feature': list(importance_dict.keys()),
    'importance': list(importance_dict.values())
}).sort_values('importance', ascending=False).head(10)

plt.figure(figsize=(12, 8))
sns.set_style('whitegrid')

ax = sns.barplot(
    data=xgBoost_importance,
    x='importance',
    y='feature',
    hue='feature',
    legend=False,
    palette='viridis'
)

for i in ax.containers:
    ax.bar_label(i, fmt='%g', label_type='edge', fontsize=10, padding=3)

plt.title('Top 10 najważniejszych cech dla XGBoost', fontsize=16)
plt.xlabel('Wartość istotności', fontsize=12)
plt.ylabel('Cecha', fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

#### CatBoost

In [ ]:
# from catboost import CatBoostClassifier
#
# def define_catboost(trial):
#     params = {
#         'iterations': trial.suggest_int('iterations', 50, 200),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1, log=True),
#         'depth': trial.suggest_int('depth', 5, 12),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 1, log=True),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 1e-3, 1, log=True),
#         'loss_function': 'MultiClass',
#         'cat_features': my_categories,
#         'random_seed': 42,
#         'logging_level': 'Silent',
#     }
#     return CatBoostClassifier(**params)
#
# def objective_catboost(trial):
#     model = define_catboost(trial)
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X, y, cv=skf, n_jobs=-1, scoring='accuracy')
#     return scores.mean()
#
# study_cat = optuna.create_study(direction='maximize', study_name='EngineerSalaryCatBoost', sampler=optuna.samplers.TPESampler())
# study_cat.optimize(objective_catboost, n_trials=100)

In [ ]:
# catBoost = define_catboost(study_cat.best_trial).fit(X_train, y_train)

In [ ]:
# study_cat.best_params

In [ ]:
# optuna.visualization.plot_optimization_history(study_cat)

In [ ]:
# optuna.visualization.plot_slice(study_cat)

In [ ]:
# optuna.visualization.plot_param_importances(study_cat)

#### Jak wypada CatBoost w poszczególnych foldach

In [ ]:
catBoost = CatBoostClassifier(**{
    "iterations": 137,
    "learning_rate": 0.10095947308778701,
    "depth": 7,
    "l2_leaf_reg": 0.01124311885824868,
    "loss_function": 'MultiClass',
    "random_seed": 42,
    "bagging_temperature": 1.0,
    "early_stopping_rounds": 3,
    'cat_features': my_categories,
    'logging_level': 'Silent'
})

In [ ]:
folds = []
stratifiedkfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i, (train_idx, val_idx) in enumerate(stratifiedkfold.split(X, y)):
    X_train_fold, X_val_fold = X.loc[train_idx, :], X.loc[val_idx, :]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    catBoost.fit(X_train_fold, y_train_fold)
    folds.append(accuracy_score(y_val_fold, catBoost.predict(X_val_fold)) * 100)

compare_skf['cat'] = folds

In [ ]:
catBoost.fit(X_train, y_train)
catBoost_pred = catBoost.predict(X_test)
print(classification_report(y_test, catBoost_pred, target_names=salary_category_encoder.inverse_transform(np.unique(y_train))))

#### Istotność cech dla CatBoost

In [ ]:
catBoost_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': catBoost.get_feature_importance()
}).sort_values(by='importance', ascending=False).head(10)

plt.figure(figsize=(12, 8))
sns.set_style('whitegrid')

ax = sns.barplot(
    data=catBoost_importance,
    x='importance',
    y='feature',
    hue='feature',
    legend=False,
    palette='viridis'
)

for i in ax.containers:
    ax.bar_label(i, fmt='%g', label_type='edge', fontsize=10, padding=3)

plt.title('Top 10 najważniejszych cech dla CatBoost', fontsize=16)
plt.xlabel('Wartość istotności', fontsize=12)
plt.ylabel('Cecha', fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

#### Prawdopodobieństwa poszczególnych modeli

In [ ]:
catBoost_pred_proba = catBoost.predict_proba(X_test)
lgb_pred_proba = lightgbm.predict_proba(X_test)
xgBoost_pred_proba = xgBoost.predict_proba(X_test)

#### Porównanie StratifiedKFold dla modeli

In [ ]:
compare_skf

In [ ]:
intresting_fold = [(train_idx, val_idx) for i, (train_idx, val_idx) in enumerate(stratifiedkfold.split(X, y)) if i == 9]
intresting_fold = intresting_fold[0]

X_train_fold, X_val_fold = X.loc[intresting_fold[0], :], X.loc[intresting_fold[1], :]
y_train_fold, y_val_fold = y[intresting_fold[0]], y[intresting_fold[1]]

In [ ]:
X_train_fold.describe()

In [ ]:
X_val_fold.describe()

In [ ]:
X_val_fold

#### StackingClassifier

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression


stacking = StackingClassifier(
    estimators=[
        ('cat', catBoost),
        ('lgb', lightgbm),
        ('xgBoost', xgBoost),
    ],
    final_estimator=LogisticRegression(
        penalty='l2',
        C=0.01,
        class_weight='balanced',
        solver='lbfgs',
        max_iter=100,
        random_state=42,
        multi_class='multinomial'
    ),
    cv=5,
    n_jobs=-1,
    passthrough=False,
    stack_method='predict_proba'
).fit(X_train, y_train)

stacking_pred = stacking.predict(X_test)
stacking_pred_proba = stacking.predict_proba(X_test)

print(classification_report(y_test, stacking_pred, target_names=salary_category_encoder.inverse_transform(np.unique(y_train))))

#### Współczynniki LogisticRegression

In [ ]:
coefficients = stacking.final_estimator_.coef_

proba_ticks = [f'cat_{salary_category_encoder.inverse_transform([i % 3])[0]}' for i in range(coefficients.shape[1])]
proba_ticks[3:6] = [f'lgb_{salary_category_encoder.inverse_transform([i % 3])[0]}' for i in range(coefficients.shape[1] // 3)]
proba_ticks[6:9] = [f'xgb_{salary_category_encoder.inverse_transform([i % 3])[0]}' for i in range(coefficients.shape[1] // 3)]

plt.figure(figsize=(12, 8))
sns.heatmap(
    coefficients,
    yticklabels=salary_category_encoder.classes_,
    xticklabels=proba_ticks,
    annot=True,
    fmt='.2f',
    cbar=False,
    cmap='viridis'
)

plt.title('Moc wpływu składowych klas modeli na ostateczne predykcje dla StackingClassifier', fontsize=16)
plt.xlabel('Klasa modeli bazowych', fontsize=12)
plt.ylabel('Ostateczna klasa', fontsize=12)

#### Wizualizacja regresji logistycznej

In [ ]:
from matplotlib.patches import Patch

log_reg_pred = pd.DataFrame(data=stacking_pred_proba, columns=salary_category_encoder.inverse_transform([0, 1, 2]))
log_reg_pred['true'] = salary_category_encoder.inverse_transform(y_test)

colors = ['red', 'green', 'blue']
classes = salary_category_encoder.inverse_transform([0, 1, 2])
palette = dict(zip(classes, colors))
g = sns.pairplot(log_reg_pred, hue='true', palette=palette)
g._legend.remove()

handles = [Patch(color=colors[i], label=cls) for i, cls in enumerate(classes)]

plt.legend(handles=handles, title='Prawdziwa klasa', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.suptitle('Predykcje regresji logistycznej, a rzeczywisty podział klas', y=1.02)
plt.show()

### <center>Macierze pomyłek</center>

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 12))

fig.suptitle('Macierze pomyłek dla wybranych klasyfikatorów', fontsize=16, fontweight='bold')

pred = [catBoost_pred, lightgbm_pred, xgBoost_pred, stacking_pred]
titles = ['CatBoost', 'LightGBM', 'XGBoost', 'StackingClassifier']

sns.set_style('white')

for i in range(len(titles)):
    xc, yc = divmod(i, 2)
    conf_matrix = confusion_matrix(y_test, pred[i])
    sns.heatmap(
        conf_matrix.T,
        annot=True,
        fmt='d',
        cbar=False,
        xticklabels=salary_category_encoder.classes_,
        yticklabels=salary_category_encoder.classes_,
        ax=ax[xc, yc],
        cmap='rocket',
        annot_kws={'size': 16}
    )
    ax[xc, yc].set_title(f'{titles[i]}', fontsize=16, pad=12)
    ax[xc, yc].set_xlabel('Rzeczywiste etykiety' if xc == 1 else '', fontdict={'fontsize': 14})
    ax[xc, yc].set_ylabel('Przewidziane etykiety' if i % 2 == 0 else '', fontdict={'fontsize': 14})
    ax[xc, yc].grid(False)


plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

### <center>Krzywe ROC</center>

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)

class_of_interest = salary_category_encoder.inverse_transform(class_labels)

fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 12))
fig.suptitle('Krzywe ROC One-vs-Rest dla wszystkich klasyfikatorów', fontsize=16, fontweight='bold')
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#17becf']

model_pred_proba = [catBoost_pred_proba, lgb_pred_proba, xgBoost_pred_proba, stacking_pred_proba]
titles = ['CatBoost', 'LightGBM', 'XGBoost', 'StackingClassifier']

for mi in range(len(titles)):
    model_auc_scores = [roc_auc_score(y_onehot_test[:, i], model_pred_proba[mi][:, i]) for i in range(len(class_labels))]
    legend_elements = [Line2D([0], [0], color=colors[i], label=f'{class_of_interest[i]} (AUC = {model_auc_scores[i]:.3f})') for i in range(len(class_of_interest))]
    legend_elements.append(Line2D([0], [0], linestyle='dashed', color='black', label=f'Losowy wybór (AUC = 0.5)'))
    for class_id in range(len(class_labels)):
        RocCurveDisplay.from_predictions(
            y_onehot_test[:, class_id],
            model_pred_proba[mi][:, class_id],
            name=f'{class_of_interest[class_id]} vs Rest',
            plot_chance_level=True,
            ax=ax[class_id, mi],
            color=colors[class_id],
        )
        ax[class_id, mi].grid(True)
        ax[class_id, mi].set_label(class_of_interest[class_id])
        ax[class_id, mi].set_title(f'{class_of_interest[class_id]} vs Rest', fontsize=14)
        ax[class_id, mi].set_xlabel('False Positive Rate')
        ax[class_id, mi].set_ylabel('True Positive Rate')
        ax[class_id, mi].get_legend().remove()

    ax[0, mi].legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 1.3), fontsize=10, title='Klasa (One-vs-Rest)', title_fontsize=11, frameon=True)
    ax[0, mi].text(0.5, 2, f'{titles[mi]}', ha='center', va='center', fontsize=17)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

### <center>Krzywe uczenia</center>

In [ ]:
# fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(18, 20), sharex=True, sharey=True)
#
# plt.suptitle('Krzywe uczenia dla wybranych klasyfikatorów', fontsize=16, fontweight='bold')
# models = [catBoost, lightgbm, xgBoost, stacking]
# titles = ['CatBoost', 'LightGBM', 'XGBoost', 'StackingClassifier']
# for i in range(len(titles)):
#     x1, y1 = divmod(i, 2)
#     LearningCurveDisplay.from_estimator(
#         estimator=models[i],
#         X=X,
#         y=y,
#         cv=5,
#         scoring='accuracy',
#         train_sizes=np.linspace(0.1, 1.0, 10),
#         line_kw= {'marker': 'o'},
#         ax=ax[x1, y1],
#         n_jobs=-1
#     )
#     ax[x1, y1].grid(True)
#     ax[x1, y1].set_title(titles[i], fontsize=16, pad=10)
#     ax[x1, y1].set_ylabel('Wartość dokładności')
#     ax[x1, y1].set_xlabel('Liczba danych zbioru treningowego' if i == len(titles) - 1 else '')
#
# plt.tight_layout(rect=[0, 0, 1, 0.97])
# plt.show()

#### Fit na X, y

In [ ]:
lightgbm.fit(X, y)
catBoost.fit(X, y)
xgBoost.fit(X, y)
stacking.fit(X, y)

### <center>Ostateczne predykcje</center>

In [ ]:
final_stacking_pred = stacking.predict(test)
final_stacking_pred_proba = stacking.predict_proba(test)

final_stacking_pred_proba

In [ ]:
def prepare_submission(predictions, name):
    if 'submissions' not in os.listdir(os.getcwd()):
        os.mkdir('submissions')
    submission = pd.DataFrame(
        data={
            'obs': test_obs,
            'salary_category': salary_category_encoder.inverse_transform(predictions)
        }
    )
    submission.to_csv(os.path.join('submissions', f'{name}.csv'), index=False)
    return submission

st = prepare_submission(final_stacking_pred, 'stacking_lgb_xgb_cat')
the_best = pd.read_csv('the_best.csv')

In [ ]:
st['salary_category'].value_counts()

In [ ]:
the_best['salary_category'].value_counts()

In [ ]:
st.rename(columns={'salary_category': 'new_sub'}, inplace=True)
the_best.rename(columns={'salary_category': 'old_sub'}, inplace=True)

compare = pd.merge(st, the_best, on='obs', how='inner')

In [ ]:
prepare_submission(lightgbm.predict(test), 'lgb')['salary_category'].value_counts()

In [ ]:
prepare_submission(xgBoost.predict(test), 'xgb')['salary_category'].value_counts()

In [ ]:
prepare_submission(catBoost.predict(test), 'cat')['salary_category'].value_counts()

In [ ]:
len(compare.loc[compare['new_sub'] != compare['old_sub'], 'obs'].to_numpy())

In [ ]:
compare.loc[compare['new_sub'] != compare['old_sub'], 'obs'].to_numpy()

In [ ]:
compare.loc[compare['new_sub'] != compare['old_sub'], :]

#### Informacje o modelach

In [ ]:
if 'models' not in os.listdir(os.getcwd()):
    os.mkdir('models')

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
with open(os.path.join('models', f'model_info_{timestamp}.txt'), 'w') as f:
    f.write(f'Test size: {test_size}\n\n')

    f.write('CatBoost Parameters:\n')
    for key, value in catBoost.get_params().items():
        f.write(f'"{key}": {value},\n')

    f.write('\nLightGBM Parameters:\n')
    for key, value in lightgbm.get_params().items():
        f.write(f'"{key}": {value},\n')

    f.write('\nXGBoost Parameters:\n')
    for key, value in xgBoost.get_params().items():
        f.write(f'"{key}": {value},\n')